<a href="https://colab.research.google.com/github/shmanth/Taxi_v3_open_ai_gym_solution_Q_learning_sarsa/blob/master/TaxiV3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Solving OpenAI Gym Environment - (Taxi-*v3*)
In this Python demo, we'll try solving the classic cab-driver problem. The purpose of this notebook is to show how to solve OpenAI Gym environments. We'll demonstrate Q-learning & SARSA on the Taxi environment.

Let's now look at the problem statement

Here, the objective is to pick up the passenger from one position and drop them off at another in minimum possible time. For this problem, we'll consider our environment to be a 5x5 grid.

 Image source: https://www.learndatasci.com/tutorials/reinforcement-q-learning-scratch-python-openai-gym/
There are 4 locations (R, G, Y, B) marked in the image. And the task is to pick up the passenger from one of the four locations and drop him off at other. There is a reward of +20 for a successful dropoff, and -1 for every timestep it takes and -10 for illegal pick-up and drop-off actions.

In [1]:
# Import routines
import numpy as np
import random
import time
import gym
from IPython.display import clear_output

### Calling Env

In [2]:
env = gym.make("Taxi-v3")# Create the env

state = env.reset()
env.render() # Helps in visualizing the environment

print("Current state is:", state)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym

Current state is: 232


Rendering:

- yellow: taxi is unoccupied
- green: taxi is occupied by a passenger
- blue: passenger
- magenta: destination
- other grids: locations

#### State Space
The state vector for this problem is (col_index, row_index, destination_locations, passenger_position) There are 5 rows, 5 columns and 4 destination locations. What about the passenger locations? 4 or 5?

If the passenger is not in cab that means he could be only at one of the four locations. But we also need to account for 1 addition state if the passenger is inside the cab. So, passenger could be at any 4+1 possible locations.

Therefore, the state space = 5x5x4x5 = 500

In [3]:
# No. of possible states
state_size= env.observation_space.n
print("state_space: ", state_size)

state_space:  500


### Action Space
At any state, the cab driver can either move in any of the four directions or it can pickup/ drop (legally or illegally)

- 0: south
- 1: north
- 2: east
- 3: west
- 4: pickup
- 5: drop

In [4]:
# No. of possible Actions
action_size= env.action_space.n
print("Action_space: ", action_size)

Action_space:  6


## Training
Let's now solve the given MDP using Q-learning & SARSA.

## Q-Learning
Q-Learning is an off-policy optimal control algorithm. It learns the Q-values by taking the next action based on the greedy policy

In [5]:
Q_table = np.zeros((state_size, action_size))
print(Q_table)

[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


In [6]:
episodes = 100000    # No. of episodes

# hyperparameters
learning_rate = 0.1 # Learning rate
gamma = 0.8         # Discount factor
epsilon = 0.1       # Exploration- exploitation Tradeoff

In [7]:
# Keeping the policy epsilon-greedy
def epsilon_greedy(state, table):
  z = np.random.random()
  if z > epsilon:
    action = np.argmax(table[state])
  else:
    action = env.action_space.sample()
  return action

In [8]:
start = time.time()
deltas= []

for episode in range(1, episodes+1):
  state = env.reset()
  done = False
  step = 0
  biggest_change = 0

  if episode % 5000 == 0:
    print("Episode: {}".format(episode))

  while not done:
    action = epsilon_greedy(state, Q_table)
    new_state, reward, done, info = env.step(action)
    oldQ_table = Q_table[state, action]
    Q_table[state, action] += learning_rate * (reward + gamma * np.max(Q_table[new_state,:])-Q_table[state, action])
    biggest_change = max(biggest_change, np.abs(Q_table[state][action]-oldQ_table))
    state = new_state
  deltas.append(biggest_change)
  if deltas[-1] < 0.00000001:
    break
  episode +=1

end= time.time()
training_time = end-start
print("Time_taken in seconds: ", training_time)
print("maximum difference", deltas[-1])

Time_taken in seconds:  7.379796981811523
maximum difference 9.834179692802536e-09


In [9]:
Q_table[33]

array([-3.60185896, -3.72711837, -3.70711057, -3.71124799, -6.62370001,
       -4.76883803])

##Testing the Q-Table
Let's know test our Q-learning agent on a different environment

In [10]:
state = env.reset()
env.render()

In [11]:
from IPython.display import clear_output
done = False
cumulative_reward = 0


while (done==False):
  best_action = np.argmax(Q_table[state,:])
  state,reward, done, info = env.step(best_action)
  cumulative_reward += reward

  time.sleep(0.5)
  clear_output(wait=True)
  env.render()
  print("episode_reward", cumulative_reward)
env.close()

episode_reward 10


### SARSA
SARSA is on-policy learning algorithm. Unlinke Q-learning, it learns the Q-values by taking the next action based on the current policy rather than a greedy policy

In [13]:
state= env.reset()
env.render()

print("Current state is:", state)

Current state is: 361


In [14]:
sarsa_table = np.zeros((state_size, action_size))

print(sarsa_table)

[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


In [15]:
episodes = 800000
start = time.time()
deltas= []

for episode in range(1,episodes+1):

  state = env.reset()
  step = 0
  done = False
  biggest_change = 0

  if episode % 10000 == 0:
    print("Current episode is:{}".format(episode))
  while not done:
    action = epsilon_greedy(state, sarsa_table)

    next_state, reward, done, info = env.step(action)

    old_sarsa_table = sarsa_table[state, action]

    sarsa_table[state, action] += learning_rate * (reward + gamma*sarsa_table[next_state, action]-sarsa_table[state, action])

    biggest_change = max(biggest_change, np.abs(sarsa_table[state][action] - old_sarsa_table))

    state = next_state

    deltas.append(biggest_change)

    if deltas[-1] < 0.00000001:
      break

    episode += 1

end = time.time()
training_time= start - end
print("Total_time_taken is: ",training_time)
print("Maximum difference", deltas[-1])

Current episode is:10000
Current episode is:20000
Current episode is:30000
Current episode is:40000
Current episode is:50000
Current episode is:60000
Current episode is:70000
Current episode is:80000
Current episode is:90000
Current episode is:100000
Current episode is:110000
Current episode is:120000
Current episode is:130000
Current episode is:140000
Current episode is:150000
Current episode is:160000
Current episode is:170000
Current episode is:180000
Current episode is:190000
Current episode is:200000
Current episode is:210000
Current episode is:220000
Current episode is:230000
Current episode is:240000
Current episode is:250000
Current episode is:260000
Current episode is:270000
Current episode is:280000
Current episode is:290000
Current episode is:300000
Current episode is:310000
Current episode is:320000
Current episode is:330000
Current episode is:340000
Current episode is:350000
Current episode is:360000
Current episode is:370000
Current episode is:380000
Current episode is:39

In [16]:
sarsa_table

array([[  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [ -5.        ,  -5.        ,  -5.        ,  -5.        ,
         -8.36557069, -48.02693172],
       [ -5.        ,  -5.        ,  -5.        ,  -5.        ,
         -8.29297897, -47.21837342],
       ...,
       [ -3.92315857,  -3.9141836 ,  -3.92315857,  -3.91002113,
         -8.31261189,  -8.31261189],
       [ -4.36316089,  -4.36390835,  -4.36316089,  -4.36767417,
         -9.96343246,  -7.46184887],
       [ -1.72872094,  -1.75563915,  -1.72872094,  -1.70001111,
         -4.80396016,  -6.59372334]])

In [17]:
sarsa_table[413]

array([ -5.        ,  -5.        ,  -5.        ,  -5.        ,
       -50.        , -49.99999999])

### Testing the SARSA table
Let's know test our SARSA agent on a different environment

In [19]:
state = env.reset()
env.render()

print("Current state is:", state)

Current state is: 242


In [23]:
from IPython.display import clear_output
cumulative_reward = 0
done = False

while not done:
  best_action = np.argmax(sarsa_table[state,:])
  state, reward, done, info = env.step(best_action)
  cumulative_reward += reward

  time.sleep(0.5)
  clear_output(wait = True)
  env.render()
  print("Episode Reward =", cumulative_reward)

Episode Reward = -1
